<a href="https://colab.research.google.com/github/chien2734/sgu_data_analyst/blob/chien/Lab2_DataAnalytics/NumpyAdvanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ADVANCED NUMPY


**Nạp thư viện**


In [1]:
import numpy as np
import numba as nb
import os # Dùng để quản lý file

# Khởi tạo bộ sinh số ngẫu nhiên
rng = np.random.default_rng()

print(f"NumPy version: {np.__version__}")
print(f"Numba version: {nb.__version__}")

NumPy version: 1.23.5
Numba version: 0.60.0


## A.7: Writing Fast NumPy Functions with Numba (Tăng tốc các hàm NumPy với Numba)

**Numba** là một trình biên dịch **Just-In-Time (JIT)** giúp dịch mã Python (và NumPy) sang mã máy (machine code) ngay tại thời điểm chạy. Kết quả là tốc độ thực thi có thể tăng lên hàng trăm lần, đôi khi còn nhanh hơn cả các hàm NumPy được viết sẵn bằng C.

### 1.Vấn đề: Vòng lặp `for` trong Python rất chậm

Hãy xem xét một hàm Python thuần túy tính toán `(x - y).mean()` bằng vòng lặp `for`.


In [2]:
# Hàm Python thuần túy
def mean_distance(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

# Tạo dữ liệu lớn để kiểm tra
x = rng.standard_normal(10_000_000)
y = rng.standard_normal(10_000_000)

print("Đo hiệu năng của hàm Python thuần túy:")
%timeit mean_distance(x, y)

print("\nĐo hiệu năng của hàm NumPy vector hóa:")
%timeit (x - y).mean()

Đo hiệu năng của hàm Python thuần túy:
1.92 s ± 27.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Đo hiệu năng của hàm NumPy vector hóa:
42.4 ms ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### 2. Giải pháp: Dùng `@numba.jit`

Chỉ cần thêm một **decorator** là `@nb.jit` vào trước hàm, Numba sẽ tự động biên dịch nó sang mã máy trong lần chạy đầu tiên.


In [3]:
@nb.jit
def mean_distance_numba(x, y):
    nx = len(x)
    result = 0.0
    count = 0
    for i in range(nx):
        result += x[i] - y[i]
        count += 1
    return result / count

# Lần chạy đầu tiên sẽ mất một chút thời gian để biên dịch
print("Đo hiệu năng của hàm đã được Numba tăng tốc:")
%timeit mean_distance_numba(x, y)

Đo hiệu năng của hàm đã được Numba tăng tốc:
7.98 ms ± 841 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Nhận xét**: Như bạn thấy, hàm Numba không chỉ nhanh hơn rất nhiều so với hàm Python thuần túy mà còn vượt qua cả hàm NumPy vector hóa. Điều này là do Numba có thể tối ưu hóa toàn bộ vòng lặp và các phép toán vào một khối mã máy hiệu quả, tránh được chi phí tạo ra các mảng trung gian.


### 3. Chế độ `nopython` và `njit`

Để đảm bảo hiệu năng cao nhất, chúng ta có thể yêu cầu Numba chạy ở chế độ **"nopython"** (`nopython=True`). Chế độ này sẽ báo lỗi nếu Numba không thể biên dịch toàn bộ hàm sang mã máy (tức là không còn phụ thuộc vào trình thông dịch của Python).

`@nb.njit` là một cách viết tắt cho `@nb.jit(nopython=True)`.

#### Tối ưu hóa với Chữ ký Kiểu (Type Signatures)

Bạn còn có thể giúp Numba tối ưu hơn nữa bằng cách cung cấp "chữ ký kiểu" (type signature) cho hàm. Điều này cho Numba biết chính xác kiểu dữ liệu đầu vào và đầu ra.

Ví dụ, hàm `mean_distance` của chúng ta có thể được viết lại bằng chính cú pháp của NumPy và vẫn được Numba tăng tốc.


In [5]:
from numba import float64, njit

@njit(float64(float64[:], float64[:]))
def mean_distance_numpy_style(x, y):
    return (x - y).mean()

print("Đo hiệu năng hàm Numba với cú pháp NumPy:")
%timeit mean_distance_numpy_style(x, y)

Đo hiệu năng hàm Numba với cú pháp NumPy:
28.4 ms ± 555 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Nhận xét**: Mặc dù bên trong hàm này không có vòng lặp for tường minh, @njit vẫn có thể nhận diện các hàm NumPy và biên dịch chúng thành một vòng lặp hợp nhất (fused loop) cực kỳ hiệu quả ở tầng mã máy.


### 4.Tạo ufunc tùy chỉnh với `@numba.vectorize`

Numba cũng cho phép bạn dễ dàng tạo ra các hàm `ufunc` (Universal Functions) của riêng mình. Các hàm này sẽ tự động được áp dụng lên từng phần tử của mảng NumPy.


In [ ]:
# Ghi rõ các chữ ký kiểu cho ufunc để hỗ trợ int64 và float64
@nb.vectorize(["int64(int64,int64)", "float64(float64,float64)"], nopython=True)
def nb_add(x, y):
    return x + y

a = np.arange(10)
print(f"Sử dụng ufunc nb_add: {nb_add(a, a)}")

print(f"Sử dụng accumulate: {nb_add.accumulate(a, 0)}")

Sử dụng ufunc nb_add: [ 0  2  4  6  8 10 12 14 16 18]
Sử dụng accumulate: [ 0  1  3  6 10 15 21 28 36 45]


**Nhận xét**: Với @vectorize, bạn không cần phải tự viết vòng lặp for. Bạn chỉ cần định nghĩa logic cho một cặp phần tử, và Numba sẽ lo phần còn lại, tạo ra một hàm có thể phát sóng (broadcast) và xử lý mảng một cách hiệu quả.


## A.8: Advanced Array Input and Output (Kỹ thuật Input/Output nâng cao)

Khi làm việc với các bộ dữ liệu cực lớn không thể tải hết vào bộ nhớ RAM, **Memory-Mapped Files (memmap)** là một giải pháp cứu cánh.

**Memmap** cho phép bạn thao tác với một file nhị phân trên đĩa **như thể nó là một mảng trong bộ nhớ**. NumPy sẽ tự động quản lý việc đọc các phần nhỏ của file khi cần thiết, thay vì đọc toàn bộ.


In [4]:
filename = 'mymmap'
shape = (10000, 10000)
dtype = np.float64

# Tạo một file memmap mới ở chế độ ghi và đọc ('w+')
mmap = np.memmap(filename, dtype=dtype, mode='w+', shape=shape)
print("Đối tượng memmap đã được tạo:")
print(mmap)

# Đây là một "view", không phải bản sao
section = mmap[:5]

section[:] = rng.standard_normal((5, 10000))
print("\nĐã ghi dữ liệu ngẫu nhiên vào 5 hàng đầu tiên.")

# Đồng bộ hóa các thay đổi từ bộ nhớ đệm (buffer) xuống file trên đĩa
mmap.flush()
print("Dữ liệu đã được flush xuống đĩa.")

Đối tượng memmap đã được tạo:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Đã ghi dữ liệu ngẫu nhiên vào 5 hàng đầu tiên.
Dữ liệu đã được flush xuống đĩa.


### **Đọc từ Memmap**

Để mở một file memmap đã tồn tại, bạn chỉ cần cung cấp `filename`, `dtype`, và `shape`.


In [ ]:
filename = 'mymmap' #
dtype = np.float64
shape = (10000, 10000)

mmap_read = np.memmap(filename, dtype=dtype, shape=shape)

print("5 hàng đầu tiên đọc từ file memmap đã lưu:")
print(mmap_read[:5])

# %xdel là một "magic command" của IPython giúp xóa biến và các tham chiếu của nó
%xdel mmap
%xdel mmap_read

# Xóa file vật lý khỏi đĩa
try:
    os.remove(filename)
    print(f"\nFile '{filename}' đã được xóa thành công.")
except OSError as e:
    print(f"Lỗi khi xóa file: {e}")

5 hàng đầu tiên đọc từ file memmap đã lưu:
[[ 0.55892394  0.41611858  1.12940627 ... -0.03818604  0.45331686
  -1.47637291]
 [-0.2409946   0.04709209  0.01203819 ... -0.270605   -1.66487881
  -0.24056536]
 [ 0.3050747  -0.17881751 -1.64675548 ... -1.35197121  0.35262594
  -0.41166427]
 [-0.81662365 -0.63352441  1.21095092 ...  0.74325194 -0.51218403
  -1.06096309]
 [-1.89479073  0.8836742   1.35844008 ...  0.79089143  0.40131958
  -0.85393979]]
NameError: name 'mmap' is not defined

File 'mymmap' đã được xóa thành công.


### **Các lựa chọn lưu trữ khác: HDF5**

Đối với các nhu cầu lưu trữ dữ liệu khoa học cực lớn và phức tạp, định dạng **HDF5 (Hierarchical Data Format)** là một tiêu chuẩn công nghiệp. Các thư viện Python như **h5py** và **PyTables** cung cấp giao diện thân thiện với NumPy để làm việc với file HDF5, hỗ trợ nén dữ liệu hiệu quả và truy vấn phức tạp.


## A.9 Performance Tips


### 1. Lý do NumPy nhanh hơn Python thuần

- NumPy dựa trên các mảng (arrays) thay vì danh sách Python → cho phép xử lý dữ liệu theo khối (vectorization).

- Thay vì dùng vòng lặp for, bạn nên dùng toán tử mảng (+, \*, np.mean(), np.sum(), …).

- NumPy được viết bằng C, nên các thao tác nội bộ chạy rất nhanh.


### 2. Mẹo tối ưu chính


- Chuyển vòng lặp Python sang thao tác mảng NumPy.

→ Thay vì:


In [5]:
import numpy as np
arr = [1, 2, 3, 4, 5]
total = 0
for x in arr:
    total += x

dùng:


In [6]:
arr = np.array(arr)
total = arr.sum()

- Dùng broadcasting thay vì lặp nhiều chiều.
  Broadcasting giúp tự động mở rộng mảng để tính toán mà không cần sao chép dữ liệu.

- Dùng slicing (cắt mảng) thay vì copy dữ liệu.

- Tận dụng ufuncs (universal functions) như np.add, np.exp, np.log vì chúng chạy nhanh hơn rất nhiều so với hàm Python.


### 3. Khi NumPy vẫn chưa đủ nhanh

Có thể dùng:

- Cython (viết code Python + C)

- C hoặc Fortran nếu cần tốc độ cực cao
  → Cython thường được khuyên dùng vì dễ tích hợp hơn.


### 4. Tầm quan trọng của bộ nhớ liên tục (Contiguous Memory)

- Tốc độ xử lý phụ thuộc vào cách dữ liệu được lưu trong bộ nhớ (theo hàng hay theo cột).

- NumPy có hai kiểu:

  - 'C' order: lưu theo hàng (row-major) → mặc định của NumPy

  - 'F' order: lưu theo cột (column-major) → giống Fortran


Ví dụ:


In [7]:
arr_c = np.ones((100, 10000), order='C')
arr_c.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [10]:
arr_c.flags.c_contiguous

True

In [8]:
arr_f = np.ones((100, 10000), order='F')
arr_f.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [9]:
arr_f.flags.f_contiguous

True

In [11]:
%timeit arr_c.sum(1)  # 444 µs
%timeit arr_f.sum(1)  # 581 µs

551 µs ± 128 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
681 µs ± 9.02 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


→ Mảng C (theo hàng) nhanh hơn vì dữ liệu liên tiếp trong bộ nhớ.


### 5. Một số lưu ý khác:

- .copy('C') hoặc .copy('F') có thể đổi kiểu sắp xếp bộ nhớ.

- Các view (cắt lát mảng) có thể mất tính liên tục, làm giảm tốc độ.

Ví dụ:


In [15]:
arr_f.copy('C').flags


  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [16]:
arr_c[:50].flags.contiguous  # False


True

In [17]:
arr_c[:, :50].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

# KẾT THÚC
